# PerfectNS demo

This notebook demonstrates the basic functionality of the PerfectNS module; for some background see the README and the [dynamic nested sampling paper (Higson, 2017a)](https://arxiv.org/abs/1704.03459).

### Running nested sampling calculations

The likelihood $\mathcal{L}(\theta)$, prior $\pi(\theta)$ and calculation settings are specified in a PerfectNSSettings object. For this example we will use a 10-dimensional sypherrically symmetric Gaussian likelihood with size $\sigma_\mathcal{L}=1$ and a Gaussian prior with size $\sigma_\mathcal{\pi}=10$.

In [3]:
import PerfectNS.settings
import PerfectNS.likelihoods as likelihoods
import PerfectNS.priors as priors

settings = PerfectNS.settings.PerfectNSSettings()
settings.likelihood = likelihoods.gaussian(likelihood_scale=1)
settings.prior = priors.gaussian(prior_scale=10)
settings.n_dim = 10

The "dynamic_goal" setting determines if dynamic nested sampling should be used and, if so, how to split the computational effort between increasing parameter estimaton accuracy and evidence calculation accuracy. dyanmic_goal=1 optimises purely for parameter estimation and dynamic_goal=0 optimises purely for calculating the Bayesian evidence $\mathcal{Z}$.

Lets try running standard nested sampling and dynamic nested sampling calculation:

In [24]:
import PerfectNS.nested_sampling as nested_sampling

# Standard nested sampling
settings.dynamic_goal = None
standard_ns_run = nested_sampling.generate_ns_run(settings)
# Dynamic nested sampling
settings.dynamic_goal = 1  # optimise for parameter estimation accuracy
dynamic_ns_run = nested_sampling.generate_ns_run(settings)

We can now make posterior inferences using the samples generated by the nested sampling calculations. Here we calculate:

1\. the log Bayesian evidence $\log \mathcal{Z}=\log \left( \int \mathcal{L}(\theta) \pi(\theta) \mathrm{d}\theta \right)$,

2\. the mean of the first parameter $\theta_1$,

3\. the second moment of the posterior distribution of $\theta_1$,

4\. the median of $\theta_1$,

5\. the 84% one-tailed credible interval on $\theta_1$.

For the Gaussian likelihood and prior we can calculate the posterior distribution analytically, so we first calculate the analytic values of each quantity for comparison and display the results in a Pandas data frame.

In [12]:
import PerfectNS.estimators as e
import PerfectNS.analyse_run as ar

estimator_list = [e.logzEstimator(),
                    e.paramMeanEstimator(),
                    e.paramSquaredMeanEstimator(),
                    e.paramCredEstimator(0.5),
                    e.paramCredEstimator(0.84)]
results = e.get_true_estimator_values(estimator_list, settings)
results.loc['standard run'] = ar.run_estimators(standard_ns_run, estimator_list)
results.loc['dynamic run'] = ar.run_estimators(dynamic_ns_run, estimator_list)
print(results)

                   logz    theta1  theta1squ  Median(theta1)  theta1c_0.84
true values  -32.264988  0.000000   0.990080        0.000000      0.989523
standard run -32.792971 -0.015337   0.978579       -0.057560      0.972399
dynamic run  -32.475509  0.022426   1.029955        0.055792      1.021711


### Estimating sampling errors

You can estimate the numerical uncertainties on these results by calculating the standard deviation of the sampling errors distributions each run using the bootstrap resampling approach described in [Higson (2017b)](https://arxiv.org/abs/1703.09701)

In [13]:
results.loc['standard unc'] = ar.run_std_bootstrap(standard_ns_run,
                                                   estimator_list,
                                                   n_simulate=200)
results.loc['dynamic unc'] = ar.run_std_bootstrap(dynamic_ns_run,
                                                  estimator_list,
                                                  n_simulate=200)
print(results.loc[['standard unc', 'dynamic unc']])

                  logz    theta1  theta1squ  Median(theta1)  theta1c_0.84
standard unc  0.323430  0.025770   0.041676        0.028560      0.045850
dynamic unc   0.303239  0.024199   0.037086        0.035173      0.033392


### Generating and analysing runs in parallel

Multiple nested sampling runs can be generated and analysed in parallel (this uses the concurrent.futures module).

In [15]:
import PerfectNS.parallelised_wrappers as pw
import PerfectNS.maths_functions as mf

# Generate 100 nested sampling runs
run_list = pw.generate_runs(settings, 100, max_workers=4)
# Calculate estimators for each run in parallel.
values = pw.func_on_runs(ar.run_estimators, run_list, estimator_list,
                           parallelise=True)
# Calculate the mean and standard deviation of calculation results
estimator_names = [est.name for est in estimator_list]
multi_run_tests = mf.get_df_row_summary(values, estimator_names)
print(multi_run_tests.loc[['mean', 'std']])

generate_runs took 18.162 seconds
func_on_runs: calculating run_estimators for 100 runs


func_on_runs took 0.146 seconds
           logz    theta1  theta1squ  Median(theta1)  theta1c_0.84
mean -32.304413  0.002038   0.996774        0.000738      0.996018
std    0.277242  0.026587   0.040898        0.029589      0.042385


### Comparing dynamic and standard nested sampling performance
  
Lets now compare the performance of dynamic and standard nested sampling, using the 10-dimensional Gaussian likelihood and prior. 

This is the same code that was used for Table 1 of the [dynamic nested sampling paper (Higson, 2017a)](https://arxiv.org/abs/1704.03459), although we only use 100 runs instead of 5000. Tables 2, 3 and 4 can also be replicated by changing the settings; for more information about the get_dynamic_results function look at its docstring.

In [19]:
import PerfectNS.results_tables as rt

dynamic_results_table = rt.get_dynamic_results(100, [0, 1],
                                               estimator_list, settings)
print(dynamic_results_table)

dynamic_goal = None


generate_runs took 14.471 seconds
func_on_runs: calculating run_estimators for 100 runs


func_on_runs took 0.167 seconds
dynamic_goal = 0


generate_runs took 7.060 seconds
func_on_runs: calculating run_estimators for 100 runs


func_on_runs took 0.179 seconds
dynamic_goal = 1


generate_runs took 37.773 seconds
func_on_runs: calculating run_estimators for 100 runs


func_on_runs took 0.157 seconds
get_dynamic_results took 59.912 seconds
                           n_samples       logz  logz_unc    theta1  \
true values                      NaN -32.264988  0.000000  0.000000   
mean standard                6078.94 -32.281655  0.031149  0.001207   
mean dynamic 0               6042.67 -32.287889  0.025312  0.005247   
mean dynamic 1               6050.94 -32.433895  0.175977  0.000449   
std standard                 6078.94   0.311489  0.022137  0.027557   
std dynamic 0                6042.67   0.253118  0.017988  0.029984   
std dynamic 1                6050.94   1.759770  0.125061  0.011151   
efficiency gain dynamic 0    6042.67   1.514395  0.304405  0.844660   
efficiency gain dynamic 1    6050.94   0.031331  0.006298  6.107021   

                           theta1_unc  theta1squ  theta1squ_unc  \
true values                  0.000000   0.990080       0.000000   
mean standard                0.002756   0.994509       0.004109   
mean dynamic 0  

Note that every second column gives an estimated numerical uncertainty on the values in the previous column.

Looking at the final row of dynamic_results_table (above), you should see that dynamic nested sampling targeted at parameter estimation (dynamic goal=1) has an efficiency gain (equivalent computational speedup) for parameter estimation (columns other than $\log \mathcal{Z}$) of factor of around 3 to 4 compared to standard nested sampling.

### Comparing bootstrap error estimates to observed distributions of results

Finally lets check if the bootstrap estimates of parameter estimation sampling errors are accurate, using a 3d Gaussian likelihood and Gaussian prior.

This is the same code that was used for Table 5 of the dynamic nested sampling paper, although we only use 100 runs instead of 5000. See the paper and the get_bootstrap_results function's docstring for more details.

In [23]:
bootstrap_results_table = rt.get_bootstrap_results(100, 200,
                                                   estimator_list, settings,
                                                   n_run_ci=50,
                                                   n_simulate_ci=200,
                                                   add_sim_method=False,
                                                   cred_int=0.95,
                                                   ninit_sep=False,
                                                   parallelise=True)
print(bootstrap_results_table)

generate_runs took 39.768 seconds
func_on_runs: calculating run_estimators for 100 runs


func_on_runs took 0.160 seconds
func_on_runs: calculating run_std_bootstrap for 100 runs


func_on_runs took 167.790 seconds
func_on_runs: calculating run_ci_bootstrap for 50 runs


func_on_runs took 425.378 seconds
get_bootstrap_results took 633.192 seconds
                              logz  logz_unc     theta1  theta1_unc  \
true values             -32.264988  0.000000   0.000000    0.000000   
repeats mean            -32.687642  0.158360  -0.000344    0.001288   
repeats std               1.583598  0.112541   0.012877    0.000915   
bs std / repeats std      0.924214  0.073169   0.984288    0.070181   
bs estimate % variation  34.889059  2.479457   5.780385    0.410794   
bs 0.95 CI              -30.360428  0.275084   0.018920    0.001583   
bs +-1std % coverage     59.000000  0.000000  71.000000    0.000000   
bs 0.95 CI % coverage    95.000000  0.000000  94.000000    0.000000   

                         theta1squ  theta1squ_unc  Median(theta1)  \
true values               0.990080       0.000000        0.000000   
repeats mean              0.991809       0.002065       -0.000851   
repeats std               0.020652       0.001468        0.014694   
bs std 

Note that every second column gives an estimated numerical uncertainty on the values in the previous column.

You should see that the ratio of the bootstrap error estimates to bootstrap_results the standard deviation of results (row 4 of bootstrap_results_table) has values close to 1 given the estimated numerical uncertainties.'